In [3]:
words = open("names.txt", 'r').read().splitlines()

In [4]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [5]:
b = {}

for w in words:
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1

In [6]:
sorted(b.items(), key=lambda x: -x[1])

[(('n', '<E>'), 6763),
 (('a', '<E>'), 6640),
 (('a', 'n'), 5438),
 (('<S>', 'a'), 4410),
 (('e', '<E>'), 3983),
 (('a', 'r'), 3264),
 (('e', 'l'), 3248),
 (('r', 'i'), 3033),
 (('n', 'a'), 2977),
 (('<S>', 'k'), 2963),
 (('l', 'e'), 2921),
 (('e', 'n'), 2675),
 (('l', 'a'), 2623),
 (('m', 'a'), 2590),
 (('<S>', 'm'), 2538),
 (('a', 'l'), 2528),
 (('i', '<E>'), 2489),
 (('l', 'i'), 2480),
 (('i', 'a'), 2445),
 (('<S>', 'j'), 2422),
 (('o', 'n'), 2411),
 (('h', '<E>'), 2409),
 (('r', 'a'), 2356),
 (('a', 'h'), 2332),
 (('h', 'a'), 2244),
 (('y', 'a'), 2143),
 (('i', 'n'), 2126),
 (('<S>', 's'), 2055),
 (('a', 'y'), 2050),
 (('y', '<E>'), 2007),
 (('e', 'r'), 1958),
 (('n', 'n'), 1906),
 (('y', 'n'), 1826),
 (('k', 'a'), 1731),
 (('n', 'i'), 1725),
 (('r', 'e'), 1697),
 (('<S>', 'd'), 1690),
 (('i', 'e'), 1653),
 (('a', 'i'), 1650),
 (('<S>', 'r'), 1639),
 (('a', 'm'), 1634),
 (('l', 'y'), 1588),
 (('<S>', 'l'), 1572),
 (('<S>', 'c'), 1542),
 (('<S>', 'e'), 1531),
 (('j', 'a'), 1473),
 (

In [7]:
import sys
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install numpy


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [8]:
import numpy
import torch

In [9]:
N = torch.zeros((27, 27), dtype=torch.int32)

In [10]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [11]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1
     

In [12]:
N[0, :]

tensor([   0, 4410, 1306, 1542, 1690, 1531,  417,  669,  874,  591, 2422, 2963,
        1572, 2538, 1146,  394,  515,   92, 1639, 2055, 1308,   78,  376,  307,
         134,  535,  929], dtype=torch.int32)

In [13]:
p = N[0].float()
p = p / p.sum()
p

tensor([0.0000, 0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273,
        0.0184, 0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029,
        0.0512, 0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290])

In [14]:
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itos[ix]

'c'

In [15]:
g = torch.Generator().manual_seed(2147483647)
p = torch.rand(3, generator=g)
p = p/p.sum()
p

tensor([0.6064, 0.3033, 0.0903])

In [16]:
torch.multinomial(p, num_samples=20, replacement=True, generator=g)

tensor([1, 1, 2, 0, 0, 2, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1])

In [17]:
P = (N+1).float()
P /= P.sum(1, keepdim=True)
P[0].sum()

tensor(1.)

In [18]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out = []
    ix = 0
    while True:
        p = P[ix]
        # p = N[ix].float()
        # p = p/p.sum()
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        # print(itos[ix])
        if ix == 0:
            break
    print(''.join(out))
        

cexze.
momasurailezitynn.
konimittain.
llayn.
ka.


In [19]:
log_likelihood = 0.0
n = 0

for w in words:
#for w in ["andrejq"]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1, ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1
    #print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

log_likelihood=tensor(-559951.5625)
nll=tensor(559951.5625)
2.4543561935424805


In [20]:
xs, ys = [], []

for w in words[:1]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)


In [21]:
xs

tensor([ 0,  5, 13, 13,  1])

In [22]:
ys

tensor([ 5, 13, 13,  1,  0])

In [29]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=27).float()
xenc

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [30]:
xenc.shape

torch.Size([5, 27])

In [33]:
W = torch.randn((27, 27))
xenc @ W

tensor([[-0.6979,  1.3736,  1.9248,  0.4786, -0.7946, -0.0057,  1.2612,  0.5652,
         -0.1827,  0.3664,  0.7981,  1.5339,  0.1179, -2.3325,  0.6400, -0.7213,
         -1.2112,  2.1829,  1.4599,  0.7973, -0.4799,  1.8328,  1.0936, -2.1281,
         -0.9530, -0.6686,  0.3092],
        [-2.1488,  0.8236,  0.9822,  0.1593, -0.7107,  0.1303, -0.9033, -0.1092,
          0.3188,  1.6704, -0.3281,  1.5911,  1.6681,  0.8389,  1.7046, -0.3884,
          0.4562, -1.0921, -0.9904,  2.0938,  0.1960,  0.2550, -1.1029,  0.0920,
          0.4627,  2.2862,  0.0474],
        [-1.4039, -0.3446,  1.4452, -1.1444,  0.7254,  1.4376,  0.3481, -0.9824,
         -1.4841,  0.4559, -0.6442, -0.8483, -0.0067,  0.1846, -0.2461, -0.3063,
          1.6015,  1.4333, -1.1692, -1.3934,  2.4905, -0.5180, -2.2233, -0.8734,
         -0.0783,  0.1915, -1.5451],
        [-1.4039, -0.3446,  1.4452, -1.1444,  0.7254,  1.4376,  0.3481, -0.9824,
         -1.4841,  0.4559, -0.6442, -0.8483, -0.0067,  0.1846, -0.2461, -0.3063

In [35]:
logits = (xenc @ W).exp()
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
probs


tensor([[1.8384e-04, 5.8028e-03, 1.0593e-01, 5.6132e-04, 1.7560e-04, 3.0211e-04,
         3.8129e-03, 6.4949e-04, 2.5710e-04, 4.7291e-04, 1.0304e-03, 1.1528e-02,
         3.4434e-04, 1.2316e-04, 7.4459e-04, 1.8174e-04, 1.5055e-04, 7.9712e-01,
         8.2829e-03, 1.0287e-03, 2.0753e-04, 5.7985e-02, 2.2118e-03, 1.2590e-04,
         1.6436e-04, 1.8658e-04, 4.3651e-04],
        [4.8981e-05, 4.2560e-04, 6.2961e-04, 1.4082e-04, 7.1244e-05, 1.3618e-04,
         6.5368e-05, 1.0684e-04, 1.7247e-04, 8.8594e-03, 8.9577e-05, 5.9080e-03,
         8.7529e-03, 4.4077e-04, 1.0660e-02, 8.5883e-05, 2.1123e-04, 6.0967e-05,
         6.3197e-05, 1.4593e-01, 1.4713e-04, 1.5843e-04, 6.0747e-05, 1.3048e-04,
         2.1339e-04, 8.1631e-01, 1.2438e-04],
        [7.3266e-06, 1.1640e-05, 3.9877e-04, 7.8796e-06, 4.5213e-05, 3.8626e-04,
         2.3625e-05, 8.3337e-06, 7.1892e-06, 2.7757e-05, 9.6889e-06, 8.7937e-06,
         1.5474e-05, 1.9079e-05, 1.2524e-05, 1.1965e-05, 8.1744e-04, 3.7933e-04,
         7.8185e-

In [36]:
probs[0].sum()

tensor(1.0000)